### Импорты

In [51]:

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pymorphy2
import joblib

In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alxmr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Загрузка датасетов

In [53]:
df_reviews = pd.read_csv('raw/russian_reviews/reviews.csv', sep='\t', encoding='utf-8')
df_toxic = pd.read_csv('raw/russian_toxic/labeled.csv', encoding='utf-8')
df_russian_q = pd.read_csv('raw/russian_q/test_data.csv', sep='\t', encoding='utf-8', header=None, names=['raw_data'])
df_russian_q

,raw_data
0,|premise|hypothesis|label|chat
1,"0|Подскажите , сколько стоит снять покрытие ге..."
2,"1|""Супер ☺️"
3,"И фото здоровское😍""|Нет. Но часто там бываю.|1..."
4,"2|нет вроде|Да не, исправил старое сообщение ж..."
...,...
10259,"8685|Ого, спасибо. Я о таких недугах почти не ..."
10260,"8686|Без вкуса он, как сухая вода 💦)|Песок;))?..."
10261,"8687|Да разные мнения есть, свои + и -, но мы ..."
10262,"8688|Девочки, привет. Нас соседи топят, дверь ..."


### Парсинг третьего датасета

In [ ]:
parsed_data = []
for line in df_russian_q['raw_data']:
    if '|' in str(line):
        parts = str(line).strip().split('|')
        if len(parts) >= 4:
            parsed_data.extend([parts[1], parts[2]])  # premise, hypothesis

df_chat = pd.DataFrame({'text': parsed_data})

### Показ данных датасетов

In [55]:
df_reviews.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [56]:
df_toxic.head(5)


,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [57]:
df_russian_q.head()

,raw_data
0,|premise|hypothesis|label|chat
1,"0|Подскажите , сколько стоит снять покрытие ге..."
2,"1|""Супер ☺️"
3,"И фото здоровское😍""|Нет. Но часто там бываю.|1..."
4,"2|нет вроде|Да не, исправил старое сообщение ж..."


### Распределение классов

In [58]:
sample_size = 6000

df_reviews_small = df_reviews.sample(n=sample_size, random_state=42)
df_toxic_small = df_toxic.sample(n=sample_size, random_state=42)
df_stack_small = df_chat.sample(n=sample_size, random_state=42)


In [59]:
# Russian Reviews
df_reviews_small['category'] = 'Средняя важность'
df_reviews_small.loc[df_reviews_small['sentiment'] == 'negative', 'category'] = 'Срочный'
df_reviews_small.loc[df_reviews_small['sentiment'] == 'positive', 'category'] = 'Не срочный'

In [60]:
# Toxic Comments
df_toxic_small['category'] = 'СПАМ/Грубый'

In [61]:
# Russian Questions
df_stack_small['category'] = 'Важный'
df_stack_small.loc[df_stack_small['text'].str.contains('не работает|ошибка|проблема', na=False), 'category'] = 'Срочный'
df_stack_small.loc[df_stack_small['text'].str.contains('предлагаю|можно|когда|планируете|хотелось|добавьте', na=False), 'category'] = 'Можно отложить'

In [62]:
df_final = pd.concat([
    df_reviews_small[['review', 'category']].rename(columns={'review': 'text'}),
    df_toxic_small[['comment', 'category']].rename(columns={'comment': 'text'}),
    df_stack_small[['text', 'category']]
])

### Слияние датасетов

In [63]:
target_size = 3000
balanced_dfs = []

for category in df_final['category'].unique():
    category_df = df_final[df_final['category'] == category]
    if len(category_df) >= target_size:
        balanced_df = category_df.sample(n=target_size, random_state=42)
    else:
        balanced_df = category_df.sample(n=target_size, random_state=42, replace=True)
    balanced_dfs.append(balanced_df)

df_balanced = pd.concat(balanced_dfs)

### Итоговый датасет

In [64]:

print(df_balanced['category'].value_counts())
len(df_balanced)
df_balanced

category
Не срочный          3000
Срочный             3000
Средняя важность    3000
СПАМ/Грубый         3000
Важный              3000
Можно отложить      3000
Name: count, dtype: int64


,text,category
73801,"хорошая футболка, на мой размер м села довольн...",Не срочный
86778,Доставка за месяц! Спасибо! С размером угадала...,Не срочный
67097,"Качество отличное. Ткань плотная, приятная к т...",Не срочный
77889,"юбочка ткань плотная,резинка тугая,длина в са...",Не срочный
87086,Очень качественная вещь и по размеру. Мне все ...,Не срочный
...,...,...
12746,"Девушки, крик души! посоветуйте хорошего дерма...",Можно отложить
1459,Если кроме температуры пока ничего нет и темпе...,Можно отложить
1878,"То есть когда ты указал в функции L = [], то в...",Можно отложить
720,"Для внутреннего применения при гриппе можно, и...",Можно отложить


In [65]:
df_balanced.to_csv('final_dataset.csv', sep='\t', index=False, encoding='utf-8')

### Предобработка текста в итоговом датасете

In [66]:
morph = pymorphy2.MorphAnalyzer()
df = pd.read_csv('final_dataset.csv', sep='\t', encoding='utf-8')

In [67]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^а-яё\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = text.split()
    stop_words = set(stopwords.words('russian'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return ' '.join(tokens)


In [68]:
df['processed_text'] = df['text'].apply(preprocess_text)

In [69]:
df

,text,category,processed_text
0,"хорошая футболка, на мой размер м села довольн...",Не срочный,хороший футболка размер село довольно плотно х...
1,Доставка за месяц! Спасибо! С размером угадала...,Не срочный,доставка месяц спасибо размер угадать смотреть...
2,"Качество отличное. Ткань плотная, приятная к т...",Не срочный,качество отличный ткань плотный приятный тело ...
3,"юбочка ткань плотная,резинка тугая,длина в са...",Не срочный,юбочка ткань плотнаярезинка тугаядлина самый р...
4,Очень качественная вещь и по размеру. Мне все ...,Не срочный,очень качественный вещь размер нравиться спаси...
...,...,...,...
17995,"Девушки, крик души! посоветуйте хорошего дерма...",Можно отложить,девушка крик душа посоветовать хороший дермато...
17996,Если кроме температуры пока ничего нет и темпе...,Можно отложить,кроме температура пока температура расти резко...
17997,"То есть когда ты указал в функции L = [], то в...",Можно отложить,указать функция последующий вызов функция прох...
17998,"Для внутреннего применения при гриппе можно, и...",Можно отложить,внутренний применение грипп ион серебро уничто...


### TfidfVectorizer, разделение на выборки, обучение модели, predict, classification_report

In [70]:
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8
)

In [71]:
X_tfidf = tfidf.fit_transform(df['processed_text'])
y = df['category']

In [72]:
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y
)

In [73]:
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
model.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   27.2s finished


RandomForestClassifier(n_jobs=-1, random_state=42, verbose=1)

In [74]:
y_pred = model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s


Accuracy: 0.8297
                  precision    recall  f1-score   support

          Важный       0.65      0.74      0.69       600
  Можно отложить       0.98      0.97      0.97       600
      Не срочный       0.91      0.93      0.92       600
     СПАМ/Грубый       0.74      0.63      0.68       600
Средняя важность       0.85      0.81      0.83       600
         Срочный       0.87      0.89      0.88       600

        accuracy                           0.83      3600
       macro avg       0.83      0.83      0.83      3600
    weighted avg       0.83      0.83      0.83      3600



[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


### Сохранение модели и векторайзера


In [75]:
joblib.dump(model, 'rf_classifier.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
print("Модель и векторайзер сохранены")

Модель и векторайзер сохранены
